# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Lại Minh Thông
- MSSV: 20127635
- Lớp: 20CLC03


# Import

In [13]:
import pandas as pd
import numpy as np
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [14]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng cách khác nếu cần
thetas=["$$𝜃_0$$", "$$𝜃_1$$", "$$𝜃_2$$", "$$𝜃_3$$", "$$𝜃_4$$", "$$𝜃_5$$", "$$𝜃_6$$", "$$𝜃_7$$", "$$𝜃_8$$", "$$𝜃_9$$"]

# Cài đặt hàm

In [15]:
# Cài đặt các hàm cần thiết ở đây

def displayThetas(lr):
    values = lr.get_params()
    display(pd.DataFrame([values]).set_axis(thetas[:len(values)], axis=1).style.hide())

class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self

    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

def rmse(y, y_hat):
    return np.mean((y.ravel() - y_hat.ravel())**2)**0.5

def kFold(k, X, y):
    # Random data
    radIdx = np.arange(0, X.shape[0])
    np.random.shuffle(radIdx)
    X = (X.T[radIdx].T).reset_index(drop=True)    
    y = (y.T[radIdx].T).reset_index(drop=True)

    # Split data into k groups
    splitIdx = (X.shape[0] / k * np.arange(1, k)).astype(int)
    X = np.array_split(X, splitIdx)
    y = np.array_split(y, splitIdx)
    
    X = np.repeat([X], k, axis=0)
    y = np.repeat([y], k, axis=0)
    
    Xtrain = X[~np.eye(k, dtype=bool)]
    Xtrain = np.concatenate(Xtrain, 0)
    Xtrain = np.array(np.array_split(Xtrain, k))
    
    ytrain = y[~np.eye(k, dtype=bool)]
    ytrain = np.concatenate(ytrain, 0)
    ytrain = np.array(np.array_split(ytrain, k))
    
    Xtest = X[np.eye(k, dtype=bool)]
    ytest = y[np.eye(k, dtype=bool)]
    

    return Xtrain, ytrain, Xtest, ytest

def aveRMSE_CV_best_feature_model(k, X, y):
    featuresNum = X.shape[1]
    Xtrain, ytrain, Xtest, ytest = kFold(k, X, y)
    
    #Evaluate the RMSE correspond to to each feature
    AveRMSE = np.zeros(featuresNum)
    for i in range(0, featuresNum):
        for j in range(0, k):
            lr = OLSLinearRegression().fit(Xtrain[j, :, i, None], ytrain[j])
            
            y_hat = lr.predict(Xtest[j, :, i, None])
            AveRMSE[i] += rmse(ytest[j], y_hat)
        AveRMSE[i] /= featuresNum
        
    return AveRMSE

def aveRMSE_CV(k, X, y):
    Xtrain, ytrain, Xtest, ytest = kFold(k, X, y)
    
    #Evaluate the RMSE for the model
    X_pinv = np.linalg.inv(Xtrain.transpose(0, 2, 1) @ Xtrain) @ Xtrain.transpose(0, 2, 1)
    weights = X_pinv @ np.expand_dims(ytrain, 2)
    y_hat = np.sum(weights.transpose(0, 2, 1) * Xtest, axis=2)
    AveRMSE = rmse(ytest, y_hat)

    return AveRMSE

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [16]:
# Phần code cho yêu cầu 1a
lr = OLSLinearRegression().fit(X_train, y_train)
displayThetas(lr)

$$𝜃_0$$,$$𝜃_1$$,$$𝜃_2$$,$$𝜃_3$$,$$𝜃_4$$,$$𝜃_5$$,$$𝜃_6$$,$$𝜃_7$$,$$𝜃_8$$,$$𝜃_9$$
0.015101,0.090220,0.042922,0.139289,-0.567333,-0.000101,0.740713,0.190936,24.505974,2.393517


In [17]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
y_hat = lr.predict(X_test)
rmse(y_test, y_hat)

7.064046430584032

Công thức hồi quy

$$\text{Life expectancy} = \theta_0*(\text{Adult Mortality}) + \theta_1*(\text{BMI}) + \theta_2*(\text{Polio}) + \theta_3*(\text{Diphtheria}) + \theta_4*(\text{HIV/AIDS}) + \theta_5*(\text{GDP}) + \theta_6*(\text{Thinness age 10-19}) + \theta_7*(\text{Thinness age 5-9}) + \theta_8*(\text{Income composition of resources}) + \theta_9*(\text{Schooling})$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [18]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
CVData = np.array([X_train.columns, aveRMSE_CV_best_feature_model(5, X_train, y_train)])
CVdf = pd.DataFrame(CVData.T, columns=["Mô hình với 1 đặc trưng", "RMSE"])
CVdf.index = pd.RangeIndex(stop=CVdf.shape[0], name="STT") + 1

SortedFeatures = CVdf.sort_values(["RMSE"], ignore_index=True)

# In ra các kết quả cross-validation như yêu cầu
print("Best Feature: ", SortedFeatures.iloc[0, 0])
print("RMSE Value: ", SortedFeatures.iloc[0, 1])
display(CVdf)

Best Feature:  Schooling
RMSE Value:  5.889208903410413


,Mô hình với 1 đặc trưng,RMSE
STT,,
1,Adult Mortality,23.10496
2,BMI,13.977037
3,Polio,9.012443
4,Diphtheria,8.001979
5,HIV/AIDS,33.542162
6,GDP,30.093772
7,Thinness age 10-19,25.918221
8,Thinness age 5-9,25.846069
9,Income composition of resources,6.624262


In [19]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
Xbest_train = X_train[SortedFeatures.iloc[[0], 0]]

lr = OLSLinearRegression().fit(Xbest_train, y_train)
pd.DataFrame([lr.get_params()]).set_axis([thetas[0]], axis=1).style.hide()
displayThetas(lr)

$$𝜃_0$$
5.557399


In [20]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model
Xbest_test = X_test[SortedFeatures.iloc[[0], 0]]

y_hat = lr.predict(Xbest_test)
rmse(y_test, y_hat)

10.26095039165537

Công thức hồi quy

$$\text{Life expectancy} = \theta*(\text{Schooling})$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

In [21]:
# Phần code cho yêu cầu 1c

# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)

# Mô hình 1 - Một đặc trưng tốt nhất mũ 2
X1_train = pd.concat([X_train[SortedFeatures.iloc[1:, 0]], X_train[SortedFeatures.iloc[0:1, 0]]**(2)], axis=1)
X1_test = pd.concat([X_test[SortedFeatures.iloc[1:, 0]], X_test[SortedFeatures.iloc[0:1, 0]]**(2)], axis=1)

# Mô hình 2 - Một đặc trưng tốt nhất mũ 1/2
X2_train = pd.concat([X_train[SortedFeatures.iloc[1:, 0]], X_train[SortedFeatures.iloc[0:1, 0]]**(1/2)], axis=1)
X2_test = pd.concat([X_test[SortedFeatures.iloc[1:, 0]], X_test[SortedFeatures.iloc[0:1, 0]]**(1/2)], axis=1)

# Mô hình 3 - Hai đặc trưng tốt nhất mũ 1/2
X3_train = pd.concat([X_train[SortedFeatures.iloc[2:, 0]], X_train[SortedFeatures.iloc[0:2, 0]]**(1/2)], axis=1)
X3_test = pd.concat([X_test[SortedFeatures.iloc[2:, 0]], X_test[SortedFeatures.iloc[0:2, 0]]**(1/2)], axis=1)

# Mô hình 4 - Hai đặc trưng tốt nhất mũ 1/4
X4_train = pd.concat([X_train[SortedFeatures.iloc[2:, 0]], X_train[SortedFeatures.iloc[0:2, 0]]**(1/4)], axis=1)
X4_test = pd.concat([X_test[SortedFeatures.iloc[2:, 0]], X_test[SortedFeatures.iloc[0:2, 0]]**(1/4)], axis=1)

ModelNames = pd.Series([
    "Tuyến tính gốc",
    "Một đặc trưng tốt nhất mũ 2",
    "Một đặc trưng tốt nhất mũ 1/2",
    "Hai đặc trưng tốt nhất mũ 1/2",
    "Hai đặc trưng tốt nhất mũ 1/4",
])

ModelData = [
    aveRMSE_CV(5, X_train, y_train), 
    aveRMSE_CV(5, X1_train, y_train), 
    aveRMSE_CV(5, X2_train, y_train),
    aveRMSE_CV(5, X3_train, y_train),
    aveRMSE_CV(5, X4_train, y_train),
]

CVModelData = np.array([ModelNames, ModelData])
CVModeldf = pd.DataFrame(CVModelData.T, columns=["Mô hình", "RMSE"])
CVModeldf.index = pd.RangeIndex(stop=CVModeldf.shape[0], name="STT")

SortedModels = CVModeldf.sort_values(["RMSE"], ignore_index=True)

# In ra các kết quả cross-validation như yêu cầu
print("Best Model: ", SortedModels.iloc[0, 0])
print("RMSE Value: ", SortedModels.iloc[0, 1])
display(CVModeldf)

Best Model:  Hai đặc trưng tốt nhất mũ 1/4
RMSE Value:  3.96740530107112


,Mô hình,RMSE
STT,,
0,Tuyến tính gốc,7.876819
1,Một đặc trưng tốt nhất mũ 2,8.865458
2,Một đặc trưng tốt nhất mũ 1/2,5.283063
3,Hai đặc trưng tốt nhất mũ 1/2,5.192647
4,Hai đặc trưng tốt nhất mũ 1/4,3.967405


In [22]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
XModels_train = [X_train, X1_train, X2_train, X3_train, X4_train]
XModels_test = [X_test, X1_test, X2_test, X3_test, X4_test]

BestModelIndx = pd.Index(ModelNames).get_loc(SortedModels.iloc[0, 0])

lr = OLSLinearRegression().fit(XModels_train[BestModelIndx], y_train)
displayThetas(lr)

$$𝜃_0$$,$$𝜃_1$$,$$𝜃_2$$,$$𝜃_3$$,$$𝜃_4$$,$$𝜃_5$$,$$𝜃_6$$,$$𝜃_7$$,$$𝜃_8$$,$$𝜃_9$$
0.027189,-0.000074,0.044603,-0.015186,-0.052223,0.006812,0.000073,-0.564032,35.210323,3.838126


In [23]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
y_hat = lr.predict(XModels_test[BestModelIndx])
rmse(y_test, y_hat)

3.8706071903223678

Công thức hồi quy

Mô hình 1 - Một đặc trưng tốt nhất mũ 2:
$$\text{Life expectancy} = \theta_0*(\text{Adult Mortality}) + \theta_1*(\text{BMI}) + \theta_2*(\text{Polio}) + \theta_3*(\text{Diphtheria}) + \theta_4*(\text{HIV/AIDS}) + \theta_5*(\text{GDP}) + \theta_6*(\text{Thinness age 10-19}) + \theta_7*(\text{Thinness age 5-9}) + \theta_8*(\text{Income composition of resources}) + \theta_9*(\text{Schooling})^2$$

Mô hình 2 - Một đặc trưng tốt nhất mũ 1/2:
$$\text{Life expectancy} = \theta_0*(\text{Adult Mortality}) + \theta_1*(\text{BMI}) + \theta_2*(\text{Polio}) + \theta_3*(\text{Diphtheria}) + \theta_4*(\text{HIV/AIDS}) + \theta_5*(\text{GDP}) + \theta_6*(\text{Thinness age 10-19}) + \theta_7*(\text{Thinness age 5-9}) + \theta_8*(\text{Income composition of resources}) + \theta_9*(\text{Schooling})^\frac{1}{2}$$

Mô hình 3 - Hai đặc trưng tốt nhất mũ 1/2:
$$\text{Life expectancy} = \theta_0*(\text{Adult Mortality}) + \theta_1*(\text{BMI}) + \theta_2*(\text{Polio}) + \theta_3*(\text{Diphtheria}) + \theta_4*(\text{HIV/AIDS}) + \theta_5*(\text{GDP}) + \theta_6*(\text{Thinness age 10-19}) + \theta_7*(\text{Thinness age 5-9}) + \theta_8*(\text{Income composition of resources})^\frac{1}{2} + \theta_9*(\text{Schooling})^\frac{1}{2}$$

Mô hình 4 - Hai Đặc trưng tốt nhất mũ 1/4:
$$\text{Life expectancy} = \theta_0*(\text{Adult Mortality}) + \theta_1*(\text{BMI}) + \theta_2*(\text{Polio}) + \theta_3*(\text{Diphtheria}) + \theta_4*(\text{HIV/AIDS}) + \theta_5*(\text{GDP}) + \theta_6*(\text{Thinness age 10-19}) + \theta_7*(\text{Thinness age 5-9}) + \theta_8*(\text{Income composition of resources})^\frac{1}{4} + \theta_9*(\text{Schooling})^\frac{1}{4}$$